In [38]:
import pandas as pd
import sklearn
import 

df = pd.read_csv("train.csv")
y_val = df.filter(["Survived"])
x_val = df.filter(["PassengerId","Pclass","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked"])
df_array = x_val.values
for i in range(len(df_array)):
    if(df_array[i][3] == "male"):
        df_array[i][3] = 0
    elif(df_array[i][3] == "female"):
        df_array[i][3] = 1


ModuleNotFoundError: No module named 'tensorflow'

In [39]:
model = tf.keras.Sequential(
    [
        layers.Dense(32, activation="relu", name="layer1"),
        layers.Dense(32, activation="relu", name="layer2"),
        layers.Dense(1,activation="sigmoid", name="layer3"),
    ]
)
y = model(df_array)
print(y)
print("hel")

NameError: name 'tf' is not defined